# 誤差逆伝播法

## 数値微分の難点

前回までで、ニューラルネットワークの基本的な学習方法である、確率的勾配降下法は実装できた

このとき、勾配の計算は数値微分によって実装していた

数値微分はシンプルに実装できて分かりやすいが、計算に時間がかかるという難点がある

そこで、ここでは、重みパラメータの勾配計算を効率良く行う**誤差逆伝播法**を実装する


## 計算グラフ

誤差逆伝播法を視覚的に理解する方法として**計算グラフ（Computational graph）**というものがある

計算グラフとは計算の過程をグラフによって表したものである

ここで言うグラフとは、データ構造としてのグラフであり、複数のノードとエッジ（ノード間を結ぶ直線）によって表現されるものである

### 計算グラフで計算問題を解く
以下のような簡単な計算問題を、計算グラフを使って解いてみる

**問:** スーパーで1個100円のりんごを2個買ったとき、支払う金額はいくらか？（消費税は10％とする）

計算グラフは以下のように、ノードごとの計算結果が左から右へ伝わるように表現する

![computational_graph01.png](./img/computational_graph01.png)

上記では、「×2」や「×1.1」を一つの演算として○（ノード）でくくっているが、「×」のような演算子は単一のノードで表現するのが望ましい

この場合、「2」と「1.1」は、それぞれ「りんごの個数」と「消費税」という変数として、ノードの外側に表記する

![computational_graph02.png](./img/computational_graph02.png)

次に、もう少し複雑な計算を計算グラフで解く

**問:** スーパーでりんご（100円／個）を2個、みかん（150円／個）を3個買ったとき、支払う金額はいくらか？（消費税は10％とする）

この問題を解くための計算グラフは以下のようになる

![computational_graph03.png](./img/computational_graph03.png)

このように、計算グラフを使って問題を解くには、

1. 計算式に使われる要素を演算子と変数に分ける
2. 演算子と変数をノードとしてエッジでつなぐ
3. 計算グラフ上で計算を左から右へ進める

という流れで作業する

ここで、「計算を左から右へ進める」処理を**順伝播（forward propagation）**と呼び、ニューラルネットワークの**推論処理**に対応する

逆に「計算を右から左へ戻る」処理を**逆伝播（back propagation）**と呼び、ニューラルネットワークの**学習処理**に対応する

この逆伝播は、この先、微分を計算するにあたって重要な働きをする